In [ ]:
# Materials configuration for heterostructure (bottom to top):
# Si (substrate) -> SiO2 -> HfO2 -> TiN

# Miller indices for each layer's surface orientation
MILLER_INDICES = [
    (1, 0, 0),  # Silicon substrate
    (1, 0, 0),  # SiO2 layer  
    (0, 0, 1),  # HfO2 layer
    (0, 0, 1),  # TiN layer
]

# Number of atomic layers for each material
THICKNESSES = [
    4,  # Silicon substrate
    3,  # SiO2 layer
    2,  # HfO2 layer  
    3,  # TiN layer
]

# Gaps between adjacent layers in Angstroms
GAPS = [
    2.5,  # Gap between Si and SiO2
    3.0,  # Gap between SiO2 and HfO2
    2.5,  # Gap between HfO2 and TiN
]

# Total vacuum at the top of the structure in Angstroms
VACUUM = 10.0


# Create High-k Metal Gate Stack Tutorial (Simplified)

This notebook demonstrates how to create a high-k metal gate stack heterostructure with four materials: Si (substrate), SiO2 (gate oxide), HfO2 (high-k dielectric), and TiN (metal gate) using the **new simplified `create_heterostructure` function**.

This approach replaces the complex multi-step interface creation with a single function call that automatically handles strain matching and layer stacking.

We'll create a representation of the material from the [QuantumATK tutorial](https://docs.quantumatk.com/tutorials/hkmg_builder/hkmg_builder.html) without the amorphous step.

<img src="https://docs.quantumatk.com/_images/update_square_default_stack.png" alt="High-k Metal Gate Stack" width="700"/>

## Key Improvements:
- **Simplified Configuration**: Replace complex `SimpleNamespace` structures with simple lists
- **Single Function Call**: Use `create_heterostructure()` instead of manual interface creation
- **Automatic Optimization**: Built-in strain matching and supercell optimization


## 1. Configuration Parameters


In [ ]:
from types import SimpleNamespace
from mat3ra.made.tools.helpers import create_heterostructure 

# Global parameters
MAX_AREA = 200  # Maximum area for strain matching
MAX_AREA_RATIO_TOL = 0.25  # Maximum area ratio tolerance for strain matching
MAX_ANGLE_TOLERANCE = 0.15  # Maximum angle tolerance for strain matching
MAX_LENGTH_TOLERANCE = 0.15  # Maximum length tolerance for strain matching

# Structure parameters for each layer
STRUCTURE_PARAMS = [
    SimpleNamespace(
        # Silicon substrate
        slab_params=SimpleNamespace(
            miller_indices=(1, 0, 0),
            thickness=4,  # atomic layers
            vacuum=5.0,  # Angstroms
            USE_ORTHOGONAL_C=True
        ),
        termination_formula=None,  # Use default termination for substrate
        interface_distance=None  # No interface for substrate
    ),
    SimpleNamespace(
        # SiO2 layer
        slab_params=SimpleNamespace(
            miller_indices=(1, 0, 0),
            thickness=3,
            vacuum=5.0,
            USE_ORTHOGONAL_C=True
        ),
        termination_formula=None,  # Use default termination for SiO2
        interface_distance=2.5,  # Distance to Si substrate
        vacuum=0.5  # Vacuum for SiO2-terminating interface layer
    ),
    SimpleNamespace(
        # HfO2 layer
        slab_params=SimpleNamespace(
            miller_indices=(0, 0, 1),
            thickness=2,
            vacuum=0.5,
            xy_supercell_matrix=[[2, 0], [0, 2]],
            USE_ORTHOGONAL_C=True
        ),
        termination_formula=None,  # Use default termination for HfO2
        interface_distance=3.0,  # Distance to SiO2
        vacuum=0.5  # Vacuum for HfO2-terminating interface layer
    ),
    SimpleNamespace(
        # TiN layer
        slab_params=SimpleNamespace(
            miller_indices=(0, 0, 1),
            thickness=3,
            vacuum=5.0,  # Add vacuum to final layer
            USE_ORTHOGONAL_C=True,
            xy_supercell_matrix=[[3, 0], [0, 3]]
        ),
        termination_formula=None,  # Use default termination for TiN
        interface_distance=2.5,  # Distance to HfO2
        vacuum=10.0  # Vacuum for TiN-terminating interface layer
    )
]

INTERFACE_1_INDEX = 11 # was found in the previous runs

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples")

### 1.3. Get input material
In this notebook we will use materials from Standata.

In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.tools.build import MaterialWithBuildMetadata

materials_1 = [
    MaterialWithBuildMetadata.create(Materials.get_by_name_first_match("SiO2")),  # SiO2
    MaterialWithBuildMetadata.create(Materials.get_by_name_first_match("Silicon")),  # Si substrate
]
materials_2 = [
    MaterialWithBuildMetadata.create(Materials.get_by_name_first_match("HfO2.*MCL")),  # HfO2
    MaterialWithBuildMetadata.create(Materials.get_by_name_first_match("TiN")),  # TiN
]

In [ ]:
# Create heterostructure using the new simplified approach
from mat3ra.made.tools.helpers import create_heterostructure
from mat3ra.standata.materials import Materials
from mat3ra.made.tools.build import MaterialWithBuildMetadata

# Get materials from Standata
crystals = [
    MaterialWithBuildMetadata.create(Materials.get_by_name_first_match("Silicon")),
    MaterialWithBuildMetadata.create(Materials.get_by_name_first_match("SiO2")),
    MaterialWithBuildMetadata.create(Materials.get_by_name_first_match("HfO2.*MCL")),
    MaterialWithBuildMetadata.create(Materials.get_by_name_first_match("TiN")),
]

# Create the heterostructure with simplified parameters
heterostructure = create_heterostructure(
    crystals=crystals,
    miller_indices=MILLER_INDICES,
    thicknesses=THICKNESSES,
    gaps=GAPS,
    vacuum=VACUUM,
    use_conventional_cell=True,
    optimize_layer_supercells=True,
)


In [ ]:
# Visualize the heterostructure
from utils.visualize import visualize_materials as visualize

visualize(
    heterostructure,
    repetitions=[1, 1, 1],
    title="Si/SiO2/HfO2/TiN Heterostructure",
    viewer="wave",
    rotation='-90x'
)


In [ ]:
# Save the heterostructure
from utils.jupyterlite import download_content_to_file, set_materials

heterostructure.name = "Si-SiO2-HfO2-TiN Heterostructure"
set_materials(heterostructure)
download_content_to_file(heterostructure.to_json(), "heterostructure_high_k_metal_gate.json")


## 2. Create Si/SiO2 Interface
### 2.1. Create Si/SiO2 Interface

In [ ]:
from mat3ra.made.tools.build.pristine_structures.two_dimensional.slab import SlabConfiguration, SlabBuilder
from utils.visualize import visualize_materials as visualize
from mat3ra.made.tools.build.compound_pristine_structures.two_dimensional.interface import \
    create_interface_zsl_between_slabs, create_interface_simple_between_slabs

film_1_params = STRUCTURE_PARAMS[1]  # SiO2 layer
substrate_1_params = STRUCTURE_PARAMS[0]  # Silicon substrate

# Create slab configurations
substrate_1_slab_config = SlabConfiguration.from_parameters(
    material_or_dict=materials_1[1],  # Silicon 
    miller_indices=substrate_1_params.slab_params.miller_indices,
    number_of_layers=substrate_1_params.slab_params.thickness,
    vacuum=0.0,
    termination_formula=substrate_1_params.termination_formula,
    use_conventional_cell=True
)

film_1_slab_config = SlabConfiguration.from_parameters(
    material_or_dict=materials_1[0],  # SiO2
    miller_indices=film_1_params.slab_params.miller_indices,
    number_of_layers=film_1_params.slab_params.thickness,
    vacuum=0.0,
    termination_formula=film_1_params.termination_formula,
    use_conventional_cell=True
)

# Create slabs
substrate_1_slab = SlabBuilder().get_material(substrate_1_slab_config)
film_1_slab = SlabBuilder().get_material(film_1_slab_config)

# Create Si/SiO2 interface using modern ZSL helper - select specified match
interface_1 = create_interface_zsl_between_slabs(
    substrate_slab=substrate_1_slab,
    film_slab=film_1_slab,
    gap=film_1_params.interface_distance,
    vacuum=film_1_params.vacuum,
    match_id=INTERFACE_1_INDEX,  # Use specified match from parameters
    max_area=MAX_AREA,
    max_area_ratio_tol=MAX_AREA_RATIO_TOL,
    max_length_tol=MAX_LENGTH_TOLERANCE,
    max_angle_tol=MAX_ANGLE_TOLERANCE,
    reduce_result_cell_to_primitive=False,
)

### 2.2. Visualize the Si/SiO2 Interface

In [ ]:
visualize(
    interface_1,
    repetitions=[1, 1, 1],
    title="Si/SiO2 Interface"
)

visualize(
    interface_1,
    repetitions=[1, 1, 1],
    title="Si/SiO2 Interface",
    rotation='-90x'
)


## 3. Add HfO2 Layer
### 3.1. Add a layer with SimpleInterfaceBuilder

In [ ]:
from mat3ra.made.tools.helpers import create_slab

film_2_params = STRUCTURE_PARAMS[2]  # HfO2 layer

# Create HfO2 slab with specified supercell matrix
film_2_slab = create_slab(
    crystal=materials_2[0],  # HfO2
    termination=None,  # Use default termination
    miller_indices=film_2_params.slab_params.miller_indices,
    number_of_layers=film_2_params.slab_params.thickness,
    vacuum=film_2_params.slab_params.vacuum,
    xy_supercell_matrix=film_2_params.slab_params.xy_supercell_matrix,
    use_orthogonal_c=film_2_params.slab_params.USE_ORTHOGONAL_C
)

interface_1_slab = create_slab(
    crystal=interface_1,  # Use the Si/SiO2 interface slab

    vacuum=0.0,  # No vacuum for interface slab
    xy_supercell_matrix=film_2_params.slab_params.xy_supercell_matrix,
    use_orthogonal_c=film_2_params.slab_params.USE_ORTHOGONAL_C
)

# Create Si/SiO2/HfO2 interface using SimpleInterfaceBuilder
interface_2 = create_interface_simple_between_slabs(
    film_slab=film_2_slab,
    substrate_slab=interface_1_slab,
    gap=film_2_params.interface_distance,  # Distance to SiO2
    vacuum=film_2_params.vacuum  # Vacuum for HfO2-terminating interface layer
)


### 3.2. Visualize the Si/SiO2/HfO2 Interface

In [ ]:
visualize(
    interface_2,
    repetitions=[1, 1, 1],
    title="Si/SiO2/HfO2 Interface"
)

visualize(
    interface_2,
    repetitions=[1, 1, 1],
    title="Si/SiO2/HfO2 Interface",
    rotation='-90x'
)

## 4. Add TiN Layer
### 4.1. Create a TiN Slab

In [ ]:

from mat3ra.made.tools.build_components.entities.reusable.three_dimensional.supercell.helpers import create_supercell

film_3_params = STRUCTURE_PARAMS[3]  # TiN layer

# Create TiN slab with specified supercell matrix
tin_slab = create_slab(
    crystal=materials_2[1],  # TiN
    termination=None,  # Use default termination
    miller_indices=film_3_params.slab_params.miller_indices,
    number_of_layers=film_3_params.slab_params.thickness,
    vacuum=film_3_params.slab_params.vacuum,
    xy_supercell_matrix=film_3_params.slab_params.xy_supercell_matrix,
    use_orthogonal_c=film_3_params.slab_params.USE_ORTHOGONAL_C
)

# Flip interface_2 for stacking TiN on top
interface_2_flipped = create_supercell(interface_2, scaling_factor=[1, -1, -1])

### 4.2. Get terminations for TiN and Si/SiO2/HfO2

In [ ]:
# TiN slab and interface_2_flipped are ready for final interface creation
print("TiN slab created successfully")
print("Si/SiO2/HfO2 interface prepared for TiN layer addition")

### 4.3. Add TiN layer with SimpleInterfaceBuilder

In [ ]:
# Create final Si/SiO2/HfO2/TiN interface configuration
interface_2_flipped_slab = create_slab(
    crystal=interface_2_flipped,  # Use the flipped Si/SiO2/HfO2 interface slab
    miller_indices=(0, 0, 1),
    number_of_layers=film_3_params.slab_params.thickness,
    vacuum=0.0,  # No vacuum for interface slab
    xy_supercell_matrix=film_3_params.slab_params.xy_supercell_matrix,
    use_orthogonal_c=film_3_params.slab_params.USE_ORTHOGONAL_C
)

interface_3 = create_interface_simple_between_slabs(
    film_slab=tin_slab,
    substrate_slab=interface_2_flipped_slab,
    gap=film_3_params.interface_distance,
    vacuum=film_3_params.vacuum
)

# Invert the interface along the z-axis to match example orientation
interface_3 = create_supercell(interface_3, scaling_factor=[1, -1, -1])

### 4.4. Visualize the Si/SiO2/HfO2/TiN Interface

In [ ]:
visualize(
    interface_3,
    title="Si/SiO2/HfO2/TiN Interface",
    viewer="wave",
    rotation='-90x'
)

## 5. Save final material


In [ ]:
from utils.jupyterlite import download_content_to_file, set_materials
interface_3.name = "Si-SiO2-HfO2-TiN Heterostructure"
set_materials(interface_3)
download_content_to_file(interface_3.to_json(), "heterostructure_high_k_metal_gate.json")